Its the mathematical perfect solution
---
So don't believe me...
----
i know a solution that has to minimize the time spend on a car has to minimize not the mean square error, but minimize the time spend on a car... Its only the solution that minimizes the spend time that will minimize the pollution. Now you have to think in 'human quality of life' or 'human way of working'. If you see you have to catchup some time, you will speed up the tests. If you see you have some time leftover, you can spend 20 seconds more on a test.... That is what is happening IMHO... Probably the best forecast of the extra time spend on a car could be the time below the average 100seconds you spend on the previous car;.. (see if i can make a predictor of that too-)

So I use the most apted method: that is doing a LP solution. LP with the complete training model as constraints... will minimize the time spend on the cars. Averaging out all constrainted solutions indead. Now as you see he clusters the predicted times in 4 groups more or less like the clustering-XGBoost is doing. The difference is that i really pick the lowest time available as constraint.



Yes don't laugh , i have a splendid LB score of -1.4
But this model has some room for improvement... so probably the LB can be alittle bit lower. I finished with a LB -8.6  and think its the right position.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb 
from sklearn.metrics import r2_score

%matplotlib inline

from IPython.display import display, HTML
# Shows all columns of a dataframe
def show_dataframe(X, rows = 2):
    display(HTML(X.to_html(max_rows=rows)))

# Datasets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')    

Making categories and combinations of categories
----

In [ ]:
#add X0-8 combinations
for xi in ['X1','X2','X3','X4','X5','X6','X8']:
    nieuwveld='X0'+xi
    train[nieuwveld]=train['X0']+'-'+train[xi]
    test[nieuwveld]=test['X0']+'-'+test[xi]

# Categorical features
cat_cols = []
for c in train.columns:
    if train[c].dtype == 'object':
        cat_cols.append(c)
print('Categorical columns:', cat_cols)

# Dublicate features
d = {}; done = []
cols = train.columns.values
for c in cols: d[c]=[]
for i in range(len(cols)):
    if i not in done:
        for j in range(i+1, len(cols)):
            if all(train[cols[i]] == train[cols[j]]):
                done.append(j)
                d[cols[i]].append(cols[j])
dub_cols = []
for k in d.keys():
    if len(d[k]) > 0: 
        # print k, d[k]
        dub_cols += d[k]        
print('Dublicates:', dub_cols)

# Constant columns
const_cols = []
for c in cols:
    if len(train[c].unique()) == 1:
        const_cols.append(c)
print('Constant cols:', const_cols)

In [ ]:
# Glue train + test
train['eval_set'] = 0; test['eval_set'] = 1
df = pd.concat([train, test], axis=0, copy=True)
# Reset index
df.reset_index(drop=True, inplace=True)

def add_new_col(x):
    if x not in new_col.keys(): 
        # set n/2 x if is contained in test, but not in train 
        # (n is the number of unique labels in train)
        # or an alternative could be -100 (something out of range [0; n-1]
        return int(len(new_col.keys())/2)
    return new_col[x] # rank of the label

for c in cat_cols:
    # get labels and corresponding means
    new_col = train.groupby(c).y.quantile(q=0.05).sort_values().to_dict()
    df[c+'new'] = df[c].apply(add_new_col)

# show the result
#show_dataframe(df, 10)
print('Shape df',df.shape)
X = df.drop(list((set(const_cols) | set(dub_cols) | set(cat_cols))), axis=1)

# Train
X_train = X[X.eval_set == 0]
y_train = X_train.pop('y'); 
X_train = X_train.drop(['eval_set', 'ID'], axis=1)
show_dataframe(X_train, 10)
# Test
X_test = X[X.eval_set == 1]
X_test = X_test.drop(['y', 'eval_set', 'ID'], axis=1)

# Base score
y_mean = -y_train.median() #q0.01 LB 0.5546 q=0.25 0.5549
# Shapes

print('Shape X_train: {}\nShape X_test: {}'.format(X_train.shape, X_test.shape))

Create LP matrix
----

 1.  that is the categories of X_train - concat a constant column
 2. append a positive X_train and a negative X_train (upper and lower limit) call it XX_train
 3. making a slack diagonal identity matrix and concat XXtrain with slack I creating XXI_train
 4. concat 2x  a negative Z identity matrix to force the solution to 0 or any target you want.
 5. create a b matrix with the 'real y-times' and the 'minimum y-times'

![matrix annotation][1]


  [1]: https://latex.codecogs.com/gif.latex?pinv%20%5Cbegin%7Bvmatrix%7D%20%5Cbegin%7Bvmatrix%7D%20aX%20%26%20.%20%26%20.%5C%5C%20.%20%26%20.%20%26%20.%5C%5C%20.%20%26%20.%20%26%20.%20%5Cend%7Bvmatrix%7D%20%26%20%5Cbegin%7Bvmatrix%7D%201b%5C%5C%201b%5C%5C%201b%20%5Cend%7Bvmatrix%7D%20%26%20%5Cbegin%7Bvmatrix%7D%201s%20%26%200%20%26%200%20%26%200%20%26%200%20%26%200%5C%5C%20%26%201s%20%26%20%26%20%26%20%26%20%5C%5C%20%26%20%26%201s%26%20%26%20%26%20%5Cend%7Bvmatrix%7D%20%26%20%5Cbegin%7Bvmatrix%7D%20-1z%5C%5C%20-1z%5C%5C%20-1z%20%5Cend%7Bvmatrix%7D%20%5C%5C%20%5Cbegin%7Bvmatrix%7D%20-aX%20%26%20.%20%26%20.%5C%5C%20.%20%26%20.%20%26%20.%5C%5C%20.%20%26%20.%20%26%20.%20%5Cend%7Bvmatrix%7D%20%26%20%5Cbegin%7Bvmatrix%7D%20-1b%5C%5C%20-1b%5C%5C%20-1b%20%5Cend%7Bvmatrix%7D%20%26%20%5Cbegin%7Bvmatrix%7D%200%20%26%200%20%26%200%20%26%201s%20%26%200%20%260%20%5C%5C%20%26%20%26%20%26%20%261s%20%26%20%5C%5C%20%26%20%26%20%26%20%26%20%26%201s%20%5Cend%7Bvmatrix%7D%20%26%20%5Cbegin%7Bvmatrix%7D%20-1z%5C%5C%20-1z%5C%5C%20-1z%20%5Cend%7Bvmatrix%7D%20%5C%5C%20%5Cbegin%7Bvmatrix%7D%200%20%26%200%20%26%200%20%5Cend%7Bvmatrix%7D%20%26%20%5Cbegin%7Bvmatrix%7D%200%20%26%20%5Cend%7Bvmatrix%7D%20%26%20%5Cbegin%7Bvmatrix%7D%200%20%26%200%20%26%200%20%26%200%20%26%200%20%26%200%20%5Cend%7Bvmatrix%7D%20%26%20%5Cbegin%7Bvmatrix%7D%200%20%26%20%5Cend%7Bvmatrix%7D%20%5Cend%7Bvmatrix%7D%20%5Ccdot%20%5Cbegin%7Bvmatrix%7D%20%5Cbegin%7Bmatrix%7D%20-y_%7Bmin%7D%5C%5C%20.%5C%5C%20.%20%5Cend%7Bmatrix%7D%5C%5C%20%5Cbegin%7Bmatrix%7D%20&plus;y_%7Bmin2%7D%5C%5C%20.%5C%5C%20.%20%5Cend%7Bmatrix%7D%20%5C%5C%20target%200%20%5Cend%7Bvmatrix%7D

In [ ]:
maxlen=1000   # change this limit ot 4000 if you have enough memory
#construct Matrix with slacks Z 
cat_cols_new = []
for c in cat_cols:
    temp=c+'new'
    cat_cols_new.append(temp)
print('New Categorical columns:', cat_cols_new)

Xc_train=X_train[:maxlen] #[cat_cols_new]
Xc_train['b_']=1.0
XXI_train = pd.concat([Xc_train.append(-Xc_train).reset_index(),   pd.DataFrame(np.identity(len(Xc_train)*2) ) ], axis=1)
X2_train=XXI_train*XXI_train

Zident=-pd.DataFrame(np.identity(len(Xc_train)),columns=[str(xz)+'z' for xz in range(len(Xc_train))] )
ZZident=Zident.append(Zident).reset_index()
XXIZZ_train= pd.concat([XXI_train,ZZident], axis=1)
print(XXIZZ_train.shape)
print(len(Xc_train)*3+16)
print(len(XXI_train)+len(Xc_train))
fv=pd.DataFrame( [0.0 for xi in range(len(Xc_train.columns)+len(Xc_train)*2+2)] + [1.0 for xi in range(len(Xc_train))],columns=['fv']) 

XXIZZ_train=XXIZZ_train.append(fv.set_index(XXIZZ_train.columns ).T).reset_index()

# the real y
b= -y_train[:maxlen].append(-y_train[:maxlen])
b= b.append(pd.DataFrame([0]) ) # find 0 point

# the minima from 'X0'
#b2 = -y_train[:maxlen].append(-X_train['X0new'][:maxlen])
#b2= b2.append(pd.DataFrame([0]) )

b2 = -X_train['X0X5new'][:maxlen].append(-X_train['X0X8new'][:maxlen])
b2= b2.append(pd.DataFrame([0]) )
#do the math



do the math
----
its the inverse of that XXIZZ matrix  and multiplication with the 'time'

In [ ]:
print(XXIZZ_train.shape)
#print(b.shape)

XXIZZ_train=XXIZZ_train.drop(['index','level_0'],axis=1)

# regression solution
solution=np.linalg.pinv(XXIZZ_train).dot(b)
# minimal solution
solution2=np.linalg.pinv(XXIZZ_train).dot(b2)

show_dataframe(pd.DataFrame(solution.T))
show_dataframe(pd.DataFrame(solution2.T))
show_dataframe(pd.DataFrame(y_train).T)
coeff=solution[:327]
coeff2=solution2[:327]
#print(coeff)
#print(coeff2)
print('slack',(solution[326:3016]).sum(),(solution[326:3016]*solution[326:3016]).sum())
print('slack2',(solution2[326:3016]).sum(),(solution2[326:3016]*solution2[326:3016]).sum())

Use the matrix to make the prediction, 
---

**look at the graph, amazing, the predictions are all sticking on the diagonal, its perfect. Thats a real good prediction.**
look at the slack... the sum of square of the slack2 is really genious low.

I stick to this solution..



In [ ]:
Xc_test=X_test #[cat_cols_new]
Xc_test['b_']=1.0
Xd_train=X_train #[cat_cols_new]
Xd_train['b_']=1.0

y_pred=-Xc_test.dot(coeff2)
y_train_pred=-Xd_train.dot(coeff2)

plt.figure(figsize=(10,10))
sns.distplot(y_train, kde=False, color='g')
sns.distplot(y_pred, kde=False, color='b')
plt.title('Distr. of train and pred. test')

plt.figure(figsize=(10,10))
plt.title('True vs. Pred. train')
plt.plot([80,265], [80,265], color='g', alpha=0.3)
plt.scatter(x=y_train, y=y_train_pred, marker='.', alpha=0.5)
plt.scatter(x=[np.mean(y_train)], y=[np.mean(y_train_pred)], marker='o', color='red')
plt.xlabel('Real train'); plt.ylabel('Pred. train')

y_pr=pd.DataFrame(y_pred).reset_index()
y_pr.columns=['index','y']
test['y_pr']=y_pr['y']

output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': test['y_pr'] })
output.to_csv('submLP.csv', index=False)


What is the room for improvement ?
----

 - The time spend on a car per category X0-X8 is not perfectly linear, outliers are in the begin and the end
see Yohan's notebook  https://www.kaggle.com/yohanb/categorical-features-encoding-xgb-0-554?scriptVersionId=1280085

 - The time spend on a car is function of ID see. SO you should 'detrend' the 'y' to make a perfect prediction
https://www.kaggle.com/plarmuseau/visualizing-the-id-trend
